In [ ]:
# Paths - Update locally!
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [ ]:
import mne
import numpy as np
import pandas as pd
import os
import re
import csv
from tqdm.notebook import tqdm
import warnings
import random
import itertools as itools

from img_pipe import img_pipe

from matplotlib import pyplot as plt
from matplotlib import rcParams as rc
import matplotlib.patheffects as PathEffects
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')
%matplotlib inline

import sys
sys.path.append(os.path.join(git_path,"analysis","mtrf"))
import mtrf_utils

In [ ]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_spkr_sn_all.txt"
        ))
    ] for s in subjs
}
models = ['model1','model2','model3','model4']

### Load mTRF results

In [ ]:
# Load corrs, best alphas from pandas
results_csv_fpath = os.path.join(git_path,"analysis","mtrf","results.csv")
df = pd.read_csv(results_csv_fpath)
# Load stim/resp from hdf5
tStims, tResps, vStims, vResps = dict(), dict(), dict(), dict()
corrs, best_alphas, ch_names = dict(), dict(), dict()
pbar = tqdm(subjs)
for s in pbar:
    tStims[s], tResps[s], vStims[s], vResps[s] = dict(), dict(), dict(), dict()
    corrs[s], best_alphas[s] = dict(), dict()
    for m in models:
        pbar.set_description(f"Loading model inputs for {s} {m}")
        # Update this file location accordingly on your local machine!
        model_input_h5_fpath = os.path.join(git_path,"analysis","mtrf" ,"h5","model_inputs",
                                            f"{s}_model_inputs.hdf5")
        tStims[s][m], tResps[s][m], vStims[s][m], vResps[s][m] = mtrf_utils.load_model_inputs(
            model_input_h5_fpath, m
        )
        print(s, m, "t/v stim:", tStims[s][m].shape, vStims[s][m].shape, "||",
              "t/v resp:", tResps[s][m].shape, vResps[s][m].shape)  
        block = [b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"preprocessing","events","csv",s)) if f"{b}_mic_sn_all.txt" in os.listdir(
            os.path.join(git_path,"preprocessing","events","csv",s,b))][0]
        blockid = "_".join([s,block])
        raw_fpath = os.path.join(data_path,f"sub-{s}",blockid,"HilbAA_70to150_8band","ecog_hilbAA70to150.fif")
        ch_names[s] = mne.io.read_raw_fif(raw_fpath,preload=True,verbose=False).info['ch_names']    
        subj_corrs, subj_best_alphas = np.zeros(len(ch_names[s])), np.zeros(len(ch_names[s]))
        for i,ch in enumerate(ch_names[s]):
            tgt_row = df[(df['subject']==s) & (df['model']==m) & (df['channel']==ch)]
            subj_corrs[i] = df.loc[tgt_row.index, 'r_value']
            subj_best_alphas[i] = df.loc[tgt_row.index, 'best_alpha']
        corrs[s][m] = np.array(subj_corrs)
        best_alphas[s][m] = np.array(subj_best_alphas)    

### Run the bootstrap

In [ ]:
delay_min, delay_max = -0.3, 0.5
delays = np.arange(np.floor(delay_min*100),np.ceil(delay_max*100),dtype=int)
pvals = dict(); nboots_shuffle = 100; chunklen = len(delays)*3 # data randomized in chunks
for s in subjs:
    pvals[s] = dict()
    for m in models:
        nsamps, nelecs = tResps[s][m].shape
        allinds = range(nsamps); nchunks = int(np.floor(0.2*nsamps/chunklen)); boot_corrs = []
        # Run the bootstrap
        pbar = tqdm(np.arange(nboots_shuffle))
        for n in pbar:
            pbar.set_description(f'{s} {m} Bootstrap {n}/{nboots_shuffle}')
            indchunks = list(zip(*[iter(allinds)]*chunklen)); random.shuffle(indchunks)
            shuff_inds = list(itools.chain(*indchunks[:nchunks]))
            tStim_shuff = tStims[s][m].copy(); tResp_shuff = tResps[s][m].copy()
            tStim_shuff = tStim_shuff[shuff_inds,:]; tResp_shuff = tResp_shuff[:len(shuff_inds),:]
            boot_corr = mtrf_utils.eigridge_corr(tStim_shuff, vStims[s][m], tResp_shuff, vResps[s][m],
                                 [best_alphas[s][m][0]], corrmin = 0.05)
            boot_corrs.append(boot_corr)
        boot_corrs = np.vstack((boot_corrs))
        # Compare bootstrap coors to STRF corrs
        # Is the correlation of the model greater than the shuffled correlation for random data?
        strf_corrs = corrs[s][m]
        h_val = np.array([strf_corrs > boot_corrs[c] for c in np.arange(len(boot_corrs))])
        print(h_val.shape) # Should be nboots x nchans
        # Count the number of times out of nboots_shuffle that the correlation is greater than 
        # random, subtract from 1 to get the bootstrapped p_val (one per electrode)
        pvals[s][m] = 1-h_val.sum(0)/nboots_shuffle

In [ ]:
# Save to csv
results_csv_fpath = os.path.join(git_path,"analysis","mtrf","results.csv")
df = pd.read_csv(results_csv_fpath)
pbar = tqdm(subjs)
for s in pbar:
    for m in models:
        pbar.set_description(f"Saving pvals for {s} {m} to csv")
        for i,ch in enumerate(ch_names[s]):
            tgt_row = df[(df['subject']==s) & (df['model']==m) & (df['channel']==ch)]
            df.loc[tgt_row.index, 'p_value'] = pvals[s][m][i]
df.to_csv(results_csv_fpath,index=False)